# Create necessary classes

## General classes

In [49]:
class Obj:
    """Obj class"""
    
    def __init__(self, project, name, nr_resources):        
        """Initialization"""
        
        self.project = project
        self.name = name
        self.resource = simpy.Resource(self.project, capacity=nr_resources)


In [50]:
class Location:
    """Location class"""
    
    def __init__(self, x, y):
        """Initialization."""        
        self.x = 0
        self.y = 0
        

In [51]:
class Container:
    """Container class"""
    
    def __init__(self, capacity, level):
        """Initialization."""
        self.container = simpy.Container(self.project, capacity, init=level)
        self.total_requested = 0
    
    def put_soil(self, amount):
        "this could be used to put specific soil characteristics"
        pass
        
    def get_soil(self, amount):
        "this could be used to get specific soil characteristics"
        pass
    

In [52]:
class Transport:
    """Transport class"""

    def __init__(self, 
                 transport_full_speed=1, transport_full_distance=1000,
                 transport_empty_speed=1.5, transport_empty_distance=1000):
        """Initialization."""
        self.transport_full_speed = transport_full_speed 
        self.transport_full_distance = transport_full_distance
        self.transport_empty_speed = transport_empty_speed 
        self.transport_empty_distance = transport_empty_distance 

    def transport_full(self):
        """transport full"""
        trip_duration = self.transport_full_distance / self.transport_full_speed

        self.log_entry('transport full start', self.project.now, self.container.level)
        yield self.project.timeout(trip_duration)
        self.log_entry('transport full stop', self.project.now, self.container.level)

    def transport_empty(self):
        """transport empty"""
        trip_duration = self.transport_empty_distance / self.transport_empty_speed

        self.log_entry('transport empty start', self.project.now, self.container.level)
        yield self.project.timeout(trip_duration)
        self.log_entry('transport empty stop', self.project.now, self.container.level)
        

In [53]:
class Processing:
    """Processing class"""
    
    def __init__(self,
                 intake_obj, output_obj, processing_rate):
        """Initialization."""
        self.intake_obj=intake_obj
        self.output_obj=output_obj
        self.processing_rate=processing_rate
        

In [54]:
class Intake:
    """Intake class"""
    
    def __init__(self,
                 intake_obj, output_obj, intake_rate):
        """Initialization."""
        self.intake_obj=intake_obj
        self.output_obj=output_obj
        self.intake_rate=intake_rate
        

In [55]:
class Output:
    """Output class"""
    
    def __init__(self,
                 intake_obj, output_obj, output_rate):
        """Initialization."""
        self.intake_obj=intake_obj
        self.output_obj=output_obj
        self.output_rate=output_rate
        

In [56]:
class Log:
    """Log class"""
    
    def __init__(self, log=[], t=[], value=[]):
        """Initialization."""
        self.log = log
        self.t = t
        self.value = value
        
    def log_entry(self, log, t, value):
        """Log"""
        self.log.append(log)
        self.t.append(t)
        self.value.append(value)
        

## Specific classes

In [57]:
class Site(Obj, Location, Container, Log):
    """Site class"""    

    def __init__(self, 
                 project, name, nr_resources=1,
                 x=0, y=0,
                 capacity=1000, level=1000, 
                 log=[], t=[], value=[]):
        """Initialization.""" 
        Obj.__init__(self, project, name, nr_resources)
        Location.__init__(self, x, y)
        Container.__init__(self, capacity, level)
        Log.__init__(self, log, t, value)
        

In [58]:
class Transport_Resource(Obj, Location, Container, Transport, Log):
    """Transport_Resource class"""    

    def __init__(self, 
                 project, name, nr_resources=1,
                 x=0, y=0,
                 capacity=1000, level=1000, 
                 full_speed=1, full_distance=1000, empty_speed=1.5, empty_distance=1000, 
                 log=[], t=[], value=[]):
        """Initialization.""" 
        Obj.__init__(self, project, name, nr_resources)
        Location.__init__(self, x, y)
        Container.__init__(self, capacity, level)
        Transport.__init__(self, full_speed, full_distance, empty_speed, empty_distance)
        Log.__init__(self, log, t, value)
        

In [59]:
class Processing_Resource(Obj, Location, Processing, Log):
    """Processing_Resource class"""    

    def __init__(self, 
                 project, name, nr_resources=1,
                 x=0, y=0,
                 intake_obj=[], output_obj=[], processing_rate=5, 
                 log=[], t=[], value=[]):
        """Initialization.""" 
        Obj.__init__(self, project, name, nr_resources)
        Location.__init__(self, x, y)
        Processing.__init__(self, intake_obj, output_obj, processing_rate)
        Log.__init__(self, log, t, value)

    def load(self, intake_obj, output_obj, value):
        intake_obj.container.get(value)
        output_obj.container.put(value)
        yield self.project.timeout(value/self.processing_rate)
        print('T=' + '{:06.2f}'.format(project.now) + ' ' + output_obj.name + ' container value: ' + str(output_obj.container.level))


In [60]:
class Transport_Processing_Resource(Obj, Location, Intake, Output, Container, Transport, Log):
    """Transport_Processing_Resource class"""    

    def __init__(self, 
                 project, name, nr_resources=1,
                 x=0, y=0,
                 capacity=1000, level=1000, 
                 intake_obj=[], intake_rate=[],
                 output_obj=[], output_rate=[],
                 full_speed=1, full_distance=1000, empty_speed=1.5, empty_distance=1000, 
                 log=[], t=[], value=[]):
        """Initialization.""" 
        Obj.__init__(self, project, name, nr_resources)
        Location.__init__(self, x, y)
        Container.__init__(self, capacity, level)
        Intake.__init__(self, intake_obj, self.container, intake_rate)
        Output.__init__(self, self.container, output_obj, output_rate)
        Transport.__init__(self, full_speed, full_distance, empty_speed, empty_distance)
        Log.__init__(self, log, t, value)
        

## Activities

In [61]:
class Installation:
    """The Installation Class forms a spefic class of activities with associated methods that can 
    initiate and suspend processes according to a number of specified conditions. This class deals 
    mainly with transport and installation of discrete objects."""
    def __init__(self, project, name, 
                 condition,
                 from_site, to_site,  
                 load_resource, transport_resource, placement_resource):
        """Initialization."""
        self.project = project
        self.name = name
        
        self.condition = condition
        
        self.from_site = from_site
        self.to_site = to_site
        
        self.load_resource = load_resource
        self.transport_resource = transport_resource
        self.placement_resource = placement_resource
        
        self.standing_by_proc = project.process(
            self.standing_by(project, 
                             condition,
                             from_site, to_site,
                             load_resource, transport_resource, placement_resource))

        self.installation_proc = project.process(
            self.installation_process_control(project,
                             condition,
                             from_site, to_site,
                             load_resource, transport_resource, placement_resource))

        self.installation_reactivate = project.event()

    def standing_by(self, project, condition,
                          from_site, to_site,
                          load_resource, transport_resource, placement_resource):
        """Standing by"""
        shown = False

        while not eval(condition):
            if not shown:
                print('T=' + '{:06.2f}'.format(project.now) + ' ' + self.name + ' to ' + to_site.name + ' suspended')
                shown = True
            yield self.project.timeout(1) # step 1 minute ahead

        print('T=' + '{:06.2f}'.format(project.now) + ' ' + 'Condition: ' + condition + ' is satisfied')

        self.installation_reactivate.succeed()  # "reactivate"
        self.installation_reactivate = self.project.event()
 
    def installation_process_control(self, project, condition,
                             from_site, to_site,
                             load_resource, transport_resource, placement_resource):
        """Installation process control"""  
        while not eval(self.condition):
            yield self.installation_reactivate

        print('T=' + '{:06.2f}'.format(project.now) + ' '+ self.name + ' to ' + to_site.name + ' started')
        while eval(condition):
            yield from self.installation_process(project, condition,
                                                 from_site, to_site,
                                                 load_resource, transport_resource, placement_resource)
        
    def installation_process(self, project, condition,
                                   from_site, to_site,
                                   load_resource, transport_resource, placement_resource):
        """Installation process."""
        delivery_volume = min(
            from_site.container.level,
            transport_resource.container.capacity - transport_resource.container.level,
            to_site.container.capacity - to_site.container.level)

        if delivery_volume>0:
            # request access to the load_resource
            with load_resource.resource.request() as my_load_resource_turn:
                yield my_load_resource_turn

                to_site.total_requested += delivery_volume
                from_site.total_requested += delivery_volume

                yield from load_resource.load(from_site, transport_resource, delivery_volume)

                load_resource.resource.release(my_load_resource_turn)

            # request access to the transport_resource
            with transport_resource.resource.request() as my_transport_resource_turn:
                yield my_transport_resource_turn

                yield from transport_resource.transport_full()

            # request access to the placement_resource
            with placement_resource.resource.request() as my_placement_resource_turn:
                yield my_placement_resource_turn

                yield from load_resource.load(transport_resource, to_site, delivery_volume)

                placement_resource.resource.release(my_placement_resource_turn)

            # request access to the transport_resource
            with transport_resource.resource.request() as my_transport_resource_turn:
                yield my_transport_resource_turn

                yield from transport_resource.transport_empty()

                transport_resource.resource.release(my_transport_resource_turn)
        
 

# Start case

In [62]:
import simpy

In [63]:
# *** Create a project environment
project = simpy.Environment()

In [64]:
# *** Define sites
from_site = Site(project, 'block_yard', 1,    0, 0, 10000, 10000)
to_site_1 = Site(project, 'site_1',            1, 1000, 0,  5000,     0)
to_site_2 = Site(project, 'site_2',            1, 2000, 0,  5000,     0)

In [65]:
# *** Define fleet
load_resource = Processing_Resource(project, 'portaal_kraan')
transport_resource_1 = Transport_Resource(project,  'barge_1', capacity = 1000, level = 0)
transport_resource_2 = Transport_Resource(project,  'barge_2', capacity = 1000, level = 0)
placement_resource = Processing_Resource(project, 'titan')

In [66]:
# *** Define activties
act_1 = Installation(project, 'block placement',
                                     '''to_site_1.container.level<5000''',
                                     from_site,  to_site_1,
                                     load_resource, transport_resource_1, placement_resource)
act_2 = Installation(project, 'block placement',\
                                     '''to_site_2.container.level<5000 and to_site_1.container.level>=3000''',\
                                     from_site,  to_site_2,\
                                     load_resource, transport_resource_2, placement_resource)


In [67]:
#*** Run the project
project.run()

T=000.00 Condition: to_site_1.container.level<5000 is satisfied
T=000.00 block placement to site_1 started
T=000.00 block placement to site_2 suspended
T=200.00 barge_1 container value: 1000
T=1400.00 site_1 container value: 1000
T=2266.67 barge_1 container value: 1000
T=3466.67 site_1 container value: 2000
T=4333.33 barge_1 container value: 1000
T=5334.00 Condition: to_site_2.container.level<5000 and to_site_1.container.level>=3000 is satisfied
T=5334.00 block placement to site_2 started
T=5533.33 site_1 container value: 3000
T=5534.00 barge_2 container value: 1000
T=6400.00 barge_1 container value: 1000
T=6734.00 site_2 container value: 1000
T=7600.00 site_1 container value: 4000
T=7600.67 barge_2 container value: 1000
T=8466.67 barge_1 container value: 1000
T=8800.67 site_2 container value: 2000
T=9666.67 site_1 container value: 5000
T=9667.33 barge_2 container value: 1000
T=10867.33 site_2 container value: 3000
T=11734.00 barge_2 container value: 1000
T=12934.00 site_2 container va

# Test space

In [68]:
import networkx as nx

In [69]:
G = nx.Graph()

In [70]:
FG = nx.Graph()
FG.add_weighted_edges_from([(1, 2, 0.125), (1, 3, 0.75), (2, 4, 1.2), (3, 4, 0.375)])
for n, nbrs in FG.adj.items():
    for nbr, eattr in nbrs.items():
        wt = eattr['weight']
        if wt < 0.5: print('(%d, %d, %.3f)' % (n, nbr, wt))

(1, 2, 0.125)
(2, 1, 0.125)
(3, 4, 0.375)
(4, 3, 0.375)


In [71]:
FG.dij

AttributeError: 'Graph' object has no attribute 'dij'